In [191]:

import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import re
import statsmodels.formula.api as smf
import altair as alt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

dir = '../00_Data_Source/'


In [192]:
# probably wont need this anymore..... since i am using STD-SUM..(cleaned by jiaxin)

def process_population_data(file_2010_2015, file_2016_2020, drop_columns=None):
    # Load input files and drop unnecessary columns
    pop_2010 = pd.read_csv(file_2010_2015, sep='\t').drop(
        ['Notes'], axis=1).dropna()
    pop_2016 = pd.read_csv(file_2016_2020, sep='\t').drop(
        ['Notes'], axis=1).dropna()
    pop = pd.concat([pop_2010, pop_2016], axis=0, ignore_index=True)
    if drop_columns:
        pop = pop.drop(drop_columns, axis=1)

    # Rename columns and process 'Race' and 'Age' columns
    pop = pop.rename(columns={'Yearly July 1st Estimates': 'Year',
                     'Ten-Year Age Groups Code': 'Age', 'States': 'State', 'Gender': 'Sex'})
    pop['Race'] = pop['Race'].str.replace('More than one race', 'Multiracial')
    pop['Race'] = pop['Race'].apply(lambda x: re.sub(' or ', '/', x))
    pop['Age'] = pop['Age'].replace(['1', '1-4', '5-14', '15-24'], '<25')
    pop['Age'] = pop['Age'].replace(['55-64', '65-74', '75-84', '85+'], '55+')
    pop['Age'] = pop['Age'].astype('category')

    # Group and aggregate data
    pop = pop.groupby(['Sex', 'Race', 'State', 'Year', 'Age']
                      ).sum().reset_index()

    return pop


population = process_population_data(
    dir + '2010-2015.txt', dir + '2016-2020.txt', ['Yearly July 1st Estimates Code', 'States Code'])


In [193]:
import pandas as pd


def clean_csv_file(dir_path):
    # Specify column names and data types
    column_names = ['Indicator', 'Geography', 'Age Group', 'Year',
                    'Cases', 'Rate per 100000', 'Race/Ethnicity', 'Population']
    column_data_types = {'Year': int, 'Cases': float, 'STD_per_100000': float}

    df = pd.read_csv(dir_path, usecols=column_names, dtype=column_data_types)

    # Rename columns
    df = df.rename(columns={'Geography': 'State', 'Age Group': 'age_group',
                   'Rate per 100000': 'STD_per_100000', 'Race/Ethnicity': 'Race'})

    # Drop duplicates
    df = df.drop_duplicates()

    # Strip leading/trailing whitespace from string columns
    df['State'] = df['State'].str.strip()
    df['age_group'] = df['age_group'].str.strip()
    df['Race'] = df['Race'].str.strip()

    # Replace missing values with column mean
    df['STD_per_100000'] = df['STD_per_100000'].fillna(
        df['STD_per_100000'].mean())
    df['Cases'] = df['Cases'].fillna(df['Cases'].mean())

    # Remove rows with invalid state names
    df = df[~df['State'].str.contains('\^')]
    return df


In [194]:


def partition_data(df, state_col, value_col, test_state, control_states, policy_year):
    """
    Splits data into pre-policy and post-policy for the test state and control states.

    Args:
        df (pandas DataFrame): the dataframe containing the data.
        state_col: column containing the state names.
        value_col: values to be analyzed.
        test_state : treatment state.
        control_states (list of str):  control states.
        policy_year (int): the year in which the policy change occurred.

    Returns:
        Four pandas DataFrames, representing the pre-policy and post-policy data for the test state and control states.
    """

    # Split the test state data into pre-policy and post-policy
    data_test = df.loc[df[state_col] ==
                       test_state, [state_col, "Year", value_col]]
    data_test['treat'] = 1
    test_pre = data_test[data_test["Year"] < policy_year]
    test_post = data_test[data_test["Year"] >= policy_year]

    # Split the control state data into pre-policy and post-policy
    data_control = df.loc[df[state_col].isin(
        control_states), [state_col, "Year", value_col]]
    data_control['treat'] = 0
    control_pre = data_control[data_control["Year"] < policy_year]
    control_post = data_control[data_control["Year"] >= policy_year]
    
    return test_pre, test_post, control_pre, control_post


In [195]:


def reg_fit(data, color, yvar, xvar, legend, ylabel, alpha=0.05):  # written by jiaxin
    colour = color

    # Filter out missing data
    x = data.loc[data[yvar].notnull(), xvar]

    # Calculate the x-axis range and step size
    xmin, xmax = x.min(), x.max()
    step = (xmax - xmin) / 100

    # Generate a grid of x-axis values for plotting
    grid = np.arange(xmin, xmax + step, step)

    # Generate predictions using the linear regression model
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    predictions = pd.DataFrame({xvar: grid})
    predictions[yvar] = model.predict(predictions[xvar])
    ci = model.conf_int(alpha=alpha)
    predictions["ci_low"] = model.get_prediction(
        predictions[xvar]).conf_int(alpha=alpha)[:, 0]
    predictions["ci_high"] = model.get_prediction(
        predictions[xvar]).conf_int(alpha=alpha)[:, 1]

    # Create a chart with the regression line and confidence interval
    predictions["Treat"] = f"{legend}"
    reg = (
        alt.Chart(predictions)
        .mark_line()
        .encode(
            x=xvar,
            y=alt.Y(yvar, title=ylabel),
            color=alt.value(colour),
            opacity=alt.Opacity("Treat", legend=alt.Legend(title="Legend"))
        )
    )
    ci = (
        alt.Chart(predictions)
        .mark_errorband(opacity=0.3)
        .encode(
            x=alt.X(xvar, title=xvar),
            y=alt.Y("ci_low", title=ylabel, scale=alt.Scale(zero=False)),
            y2="ci_high",
            color=alt.value(colour)
        )
    )
    chart = ci + reg

    return predictions, chart


def plotting_chart(data, xvar, yvar, legend, policy_year, color, ylabel, alpha=0.05):
    """Plot a chart with the data and a vertical rule at the policy year."""
    years = list(np.arange(data[xvar].min(), data[xvar].max() + 1))
    pol_year = [int(policy_year)]

    # Plot chart
    fit, reg_chart = reg_fit(
        color=color, data=data, yvar=yvar, xvar=xvar, legend=legend, ylabel=ylabel, alpha=alpha
    )
    policy = pd.DataFrame({"Year": pol_year})

    rule = (
        alt.Chart(policy)
        .mark_rule(color="black")
        .encode(alt.X("Year:Q", title="Year"))
    )
    return (reg_chart + rule).properties(width=500, height=500)


In [196]:
clean_sp = clean_csv_file(dir + 'state_output_2.csv')
grouped_df = clean_sp.groupby(['State', 'Year'])[
    'Cases', 'STD_per_100000'].sum().reset_index()


#### DD ~ New York 

In [197]:

# Set the treatment state and control states
treat_state = "New York"
control_states = ["Florida", "Louisiana", "Texas", "California", "Illinois"]
total_std = partition_data(
    grouped_df, "State", "STD_per_100000", treat_state, control_states, 2016)


In [198]:
pre_ny = plotting_chart(total_std[0], "Year", "STD_per_100000",
                        "New York", 2016, "Blue", "STD Rate per 100,000")
post_ny = plotting_chart(
    total_std[1], "Year", "STD_per_100000", "New York", 2016, "Blue", "STD Rate per 100,000")
pre_control = plotting_chart(
    total_std[2], "Year", "STD_per_100000", "Control FL,PA,TX", 2016, "#4682B4", "STD Rate per 100,000")
post_control = plotting_chart(
    total_std[3], "Year", "STD_per_100000", "Control FL,PA,TX", 2016, " #4682B4", "STD Rate per 100,000")

final = pre_ny + post_ny + pre_control + post_control
final.properties(
    title="Difference in Difference Analysis of Policy Based on Total STD Rate in New York vs Control States"
)


alt.LayerChart(...)

## Gonorrhea only

In [199]:
# Gonorrhea in New York vs Control States

Gonorrhea_only = clean_sp[clean_sp['Indicator'] == 'Gonorrhea']

# partition data based on the same treatment and control states
gonorrhea_std_ny = partition_data(
    Gonorrhea_only, "State", "STD_per_100000", treat_state, control_states, 2016)

pre_ny_gonorrhea = plotting_chart(gonorrhea_std_ny[0], "Year", "STD_per_100000",
                                  "New York", 2016, "Blue", "STD Rate per 100,000")
post_ny_gonorrhea = plotting_chart(
    gonorrhea_std_ny[1], "Year", "STD_per_100000", "New York", 2016, "Blue", "STD Rate per 100,000")
pre_control_gonorrhea = plotting_chart(
    gonorrhea_std_ny[2], "Year", "STD_per_100000", "Control States", 2016, "#4682B4", "STD Rate per 100,000")
post_control_gonorrhea = plotting_chart(
    gonorrhea_std_ny[3], "Year", "STD_per_100000", "Control States", 2016, " #4682B4", "STD Rate per 100,000")

final_gonorrhea = pre_ny_gonorrhea + post_ny_gonorrhea + \
    pre_control_gonorrhea + post_control_gonorrhea
final_gonorrhea.properties(
    title="Difference in Difference Analysis of Policy Based on Gonorrhea STD Rate in New York vs Control States"
)


alt.LayerChart(...)

## Chlamydia	

In [200]:
# Chlamydia in New York vs Control States

Chlamydia_only = clean_sp[clean_sp['Indicator'] == 'Chlamydia']

# partition data based on the same treatment and control states
chlamydia_std_ny = partition_data(
    Chlamydia_only, "State", "STD_per_100000", treat_state, control_states, 2016)

pre_ny_chlamydia = plotting_chart(chlamydia_std_ny[0], "Year", "STD_per_100000",
                                  "New York", 2016, "Blue", "STD Rate per 100,000")
post_ny_chlamydia = plotting_chart(
    chlamydia_std_ny[1], "Year", "STD_per_100000", "New York", 2016, "Blue", "STD Rate per 100,000")
pre_control_chlamydia = plotting_chart(
    chlamydia_std_ny[2], "Year", "STD_per_100000", "Control States", 2016, "#4682B4", "STD Rate per 100,000")
post_control_chlamydia = plotting_chart(
    chlamydia_std_ny[3], "Year", "STD_per_100000", "Control States", 2016, " #4682B4", "STD Rate per 100,000")

final_chlamydia = pre_ny_chlamydia + post_ny_chlamydia + \
    pre_control_chlamydia + post_control_chlamydia
final_chlamydia.properties(
    title="Difference in Difference Analysis of Policy Based on Chlamydia STD Rate in New York vs Control States"
)


alt.LayerChart(...)

## Maryland


In [201]:
treat_state2 = "Maryland"
control_states2 = ["Virginia", "Arizona", "Ohio",
                   "New Jersey", "Pennsylvania", "Delaware"]

maryland_std = partition_data(
    grouped_df, "State", "STD_per_100000", treat_state2, control_states2, 2015)

pre_md = plotting_chart(maryland_std[0], "Year", "STD_per_100000",
                        "Maryland", 2015, "Blue", "STD Rate per 100,000")
post_md = plotting_chart(
    maryland_std[1], "Year", "STD_per_100000", "Maryland", 2015, "Blue", "STD Rate per 100,000")
pre_control_md = plotting_chart(
    maryland_std[2], "Year", "STD_per_100000", "Control States", 2015, "#4682B4", "STD Rate per 100,000")
post_control_md = plotting_chart(
    maryland_std[3], "Year", "STD_per_100000", "Control States", 2015, " #4682B4", "STD Rate per 100,000")

final_md = pre_md + post_md + pre_control_md + post_control_md
final_md.properties(
    title="Difference in Difference Analysis of Policy Based on Total STD Rate in Maryland vs Control States"
)


alt.LayerChart(...)

In [202]:

gonorrhea_md = partition_data(
    Gonorrhea_only, "State", "STD_per_100000", treat_state2, control_states2, 2015)

pre_md_gonorrhea = plotting_chart(gonorrhea_md[0], "Year", "STD_per_100000",
                                  "Maryland", 2015, "Blue", "STD Rate per 100,000")
post_md_gonorrhea = plotting_chart(
    gonorrhea_md[1], "Year", "STD_per_100000", "Maryland", 2015, "Blue", "STD Rate per 100,000")
pre_control_md_gonorrhea = plotting_chart(
    gonorrhea_md[2], "Year", "STD_per_100000", "Control States", 2015, "#4682B4", "STD Rate per 100,000")
post_control_md_gonorrhea = plotting_chart(
    gonorrhea_md[3], "Year", "STD_per_100000", "Control States", 2015, " #4682B4", "STD Rate per 100,000")

final_md_gonorrhea = pre_md_gonorrhea + post_md_gonorrhea + \
    pre_control_md_gonorrhea + post_control_md_gonorrhea
final_md_gonorrhea.properties(
    title="Difference in Difference Analysis of Policy Based on Gonorrhea Rate in Maryland vs Control States"
)


alt.LayerChart(...)

In [203]:
chlamydia_md = partition_data(
    Chlamydia_only, "State", "STD_per_100000", treat_state2, control_states2, 2015)

pre_md_chlamydia = plotting_chart(chlamydia_md[0], "Year", "STD_per_100000",
                                  "Maryland", 2015, "Blue", "STD Rate per 100,000")
post_md_chlamydia = plotting_chart(
    chlamydia_md[1], "Year", "STD_per_100000", "Maryland", 2015, "Blue", "STD Rate per 100,000")
pre_control_md_chlamydia = plotting_chart(
    chlamydia_md[2], "Year", "STD_per_100000", "Control States", 2015, "#4682B4", "STD Rate per 100,000")
post_control_md_chlamydia = plotting_chart(
    chlamydia_md[3], "Year", "STD_per_100000", "Control States", 2015, " #4682B4", "STD Rate per 100,000")

final_md_chlamydia = pre_md_chlamydia + post_md_chlamydia + \
    pre_control_md_chlamydia + post_control_md_chlamydia
final_md_chlamydia.properties(
    title="Difference in Difference Analysis of Policy Based on Chlamydia STD Rate in Maryland vs Control States"
)


alt.LayerChart(...)

## Georgia total STD

In [204]:
treat_state3 = "Georgia"
control_states3 = ["North Carolina", "Mississippi", "Nevada",
                   "Arkansas", "Tennessee", "New Mexico", "Missouri"]

georgia_std = partition_data(
    grouped_df, "State", "STD_per_100000", treat_state3, control_states3, 2017)

pre_ga = plotting_chart(georgia_std[0], "Year", "STD_per_100000",
                        "Georgia", 2017, "Blue", "STD Rate per 100,000")
post_ga = plotting_chart(
    georgia_std[1], "Year", "STD_per_100000", "Georgia", 2017, "Blue", "STD Rate per 100,000")
pre_control_ga = plotting_chart(
    georgia_std[2], "Year", "STD_per_100000", "Control States", 2017, "#4682B4", "STD Rate per 100,000")
post_control_ga = plotting_chart(
    georgia_std[3], "Year", "STD_per_100000", "Control States", 2017, " #4682B4", "STD Rate per 100,000")

final_ga = pre_ga + post_ga + pre_control_ga + post_control_ga
final_ga.properties(
    title="Difference in Difference Analysis of Policy Based on Total STD Rate in Georgia vs Control States"
)


alt.LayerChart(...)

## Georgia Gonorrhea

In [205]:
# georgia gonorrhea
gonorrhea_ga = partition_data(
    Gonorrhea_only, "State", "STD_per_100000", treat_state3, control_states3, 2017)

pre_ga_gonorrhea = plotting_chart(gonorrhea_ga[0], "Year", "STD_per_100000",
                                  "Georgia", 2017, "Blue", "STD Rate per 100,000")
post_ga_gonorrhea = plotting_chart(
    gonorrhea_ga[1], "Year", "STD_per_100000", "Georgia", 2017, "Blue", "STD Rate per 100,000")
pre_control_ga_gonorrhea = plotting_chart(
    gonorrhea_ga[2], "Year", "STD_per_100000", "Control States", 2017, "#4682B4", "STD Rate per 100,000")
post_control_ga_gonorrhea = plotting_chart(
    gonorrhea_ga[3], "Year", "STD_per_100000", "Control States", 2017, " #4682B4", "STD Rate per 100,000")

final_ga_gonorrhea = pre_ga_gonorrhea + post_ga_gonorrhea + \
    pre_control_ga_gonorrhea + post_control_ga_gonorrhea
final_ga_gonorrhea.properties(
    title="Difference in Difference Analysis of Policy Based on Gonorrhea Rate in Georgia vs Control States"
)


alt.LayerChart(...)

## Georgia Chlamydia

In [206]:
# chamlydia georgia
chlamydia_ga = partition_data(
    Chlamydia_only, "State", "STD_per_100000", treat_state3, control_states3, 2017)

pre_ga_chlamydia = plotting_chart(chlamydia_ga[0], "Year", "STD_per_100000",
                                  "Georgia", 2017, "Blue", "STD Rate per 100,000")
post_ga_chlamydia = plotting_chart(
    chlamydia_ga[1], "Year", "STD_per_100000", "Georgia", 2017, "Blue", "STD Rate per 100,000")
pre_control_ga_chlamydia = plotting_chart(
    chlamydia_ga[2], "Year", "STD_per_100000", "Control States", 2017, "#4682B4", "STD Rate per 100,000")
post_control_ga_chlamydia = plotting_chart(
    chlamydia_ga[3], "Year", "STD_per_100000", "Control States", 2017, " #4682B4", "STD Rate per 100,000")

final_ga_chlamydia = pre_ga_chlamydia + post_ga_chlamydia + \
    pre_control_ga_chlamydia + post_control_ga_chlamydia
final_ga_chlamydia.properties(
    title="Difference in Difference Analysis of Policy Based on Chlamydia STD Rate in Georgia vs Control States"
)


alt.LayerChart(...)